In [55]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
model = ChatGroq(model='qwen/qwen3-32b')
model.invoke('What is the capital of France?')

AIMessage(content="<think>\nOkay, so I need to find out what the capital of France is. Hmm, I remember from school that France is a country in Europe, and I think their capital is a well-known city. Let me think... Paris comes to mind. Wait, isn't Paris famous for the Eiffel Tower and the Louvre Museum? Yeah, that sounds right. But wait, I should make sure I'm not mixing up countries. For example, the capital of Germany is Berlin, and Spain is Madrid. France... I'm pretty sure it's Paris. Maybe I can think of some other clues. The French government is based there, right? They have the Elysée Palace, which is the official residence of the president. Yeah, that's in Paris. I don't think it's Lyon or Marseille or any other big cities. Let me see... If I recall correctly, in movies and TV shows, when they show France, they often show Paris. So I'm pretty confident that Paris is the capital. But just to be thorough, is there any chance it's a different city? I can't think of any other city 

In [62]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [58]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [59]:
import os
document_path = os.path.join(os.getcwd(),'data','sample.pdf')
loader = PyPDFLoader(document_path)
documents = loader.load()
len(documents)

21

In [60]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 150, length_function=len)
docs = text_splitter.split_documents(documents)
len(docs)
docs[1].metadata

{'producer': 'pikepdf 8.15.1',
 'creator': 'arXiv GenPDF (tex2pdf:e76afa9)',
 'creationdate': '',
 'author': 'Muhammad Maaz; Liam DeVoe; Zac Hatfield-Dodds; Nicholas Carlini',
 'doi': 'https://doi.org/10.48550/arXiv.2510.09907',
 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1',
 'title': 'Agentic Property-Based Testing: Finding Bugs Across the Python Ecosystem',
 'trapped': '/False',
 'arxivid': 'https://arxiv.org/abs/2510.09907v1',
 'source': '/Users/satish/Documents/new_vision/projects/document_portal/notebook/data/sample.pdf',
 'total_pages': 21,
 'page': 0,
 'page_label': '1'}

In [66]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs,embedding_model)
retriever = vectorstore.as_retriever()
retriever.invoke('What is agentic ai?')

GoogleGenerativeAIError: Error embedding content (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 100, model: gemini-embedding-1.0\nPlease retry in 5.279816104s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/embed_content_free_tier_requests', 'quotaId': 'EmbedContentRequestsPerMinutePerUserPerProjectPerModel-Fr

In [72]:
prompt_template = """
    Answer the question based on the context provided below.
    If the context does not contain sufficient information, respond with:
    "I do not have enough information about this."
    
    Context: {context}
    
    Question: {question}
"""

In [73]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template=prompt_template,input_variables=["context","question"])
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    Answer the question based on the context provided below.\n    If the context does not contain sufficient information, respond with:\n    "I do not have enough information about this."\n    \n    Context: {context}\n    \n    Question: {question}\n')

In [74]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [ ]:
rag_chain = ({"context":retriever | format_docs, "question":RunnablePassrthrough()}|prompt | model | parser)
rag_chain.invoke("Agentic AI")

NameError: name 'retiever' is not defined